In [1]:
import pandas as pd

data = pd.DataFrame()
vocabs = []
with open('./vocab.txt', 'r') as f:
    for line in f:
        vocabs.append(line.split()[0])

rows = []
count = []
with open('./ap.dat', 'r') as f:
    for line in f:
        tmp = line.split(' ')
        count.append(tmp[0])
        tmp = tmp[1:]
        row = [0] * len(vocabs)
        for elem in tmp:
            index, value = elem.split(':')
            row[int(index)] = int(value)
        rows.append(row)
            
            
            
data = pd.DataFrame(rows, columns = vocabs)
data

,i,new,percent,people,year,two,million,president,last,government,...,closes,huts,privileges,adverse,elliot,clyde,ski,leaf,spilled,buffs
0,1,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,3,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,0,3,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,3,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2242,0,10,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2243,1,1,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2244,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from scipy.stats import dirichlet
from scipy.stats import multinomial
import numpy as np

def sampleKClasses(k, size):
    alpha = np.ones(k)
    dirc = dirichlet.rvs(alpha, size=1)
    mult = multinomial.rvs(n = 1, p = dirc[0], size = size)
    ret = []
    for elem in mult:
        for idx, vec in enumerate(elem):
            if vec == 1:
                ret.append(idx)

    return mult, ret
    
    

In [3]:
# Number of Classes 
K = 10

one_hot, Z = sampleKClasses(K, len(data))
data['Z'] = Z
data

,i,new,percent,people,year,two,million,president,last,government,...,huts,privileges,adverse,elliot,clyde,ski,leaf,spilled,buffs,Z
0,1,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,1,1,1,0,3,1,0,1,...,0,0,0,0,0,0,0,0,0,1
2,9,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,0,2,0,0,3,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,5
4,0,1,0,3,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2242,0,10,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2243,1,1,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
2244,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [4]:
word_count_per_class = data.groupby('Z').sum()
word_count_per_class

,i,new,percent,people,year,two,million,president,last,government,...,closes,huts,privileges,adverse,elliot,clyde,ski,leaf,spilled,buffs
Z,,,,,,,,,,,,,,,,,,,,,
0,43,45,34,31,44,31,43,33,28,25,...,0,0,1,0,0,0,0,0,0,0
1,579,520,571,457,456,412,477,372,355,380,...,4,0,1,0,6,2,0,0,3,1
2,279,275,258,240,211,227,197,231,216,188,...,0,2,1,0,0,0,0,2,2,0
3,424,382,361,306,283,326,258,258,264,296,...,0,3,2,1,0,1,0,1,0,2
4,58,72,51,33,35,35,16,58,31,25,...,0,1,0,0,0,0,0,0,0,1
5,583,607,505,505,461,465,486,465,457,416,...,2,0,0,3,0,3,5,3,1,2
6,20,22,12,23,9,9,21,16,11,25,...,0,0,0,0,0,0,0,0,0,0
7,41,49,108,17,35,31,41,20,36,12,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,12,5,4,1,0,2,3,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def get_probdist_per_class(wcpc):
    ret = []
    for rows in wcpc.iterrows():
        ret.append(rows[1].values)
    ret = np.array(ret)
    sum = np.sum(ret, axis=1).reshape((-1,1))
    ret = ret/sum
    return ret

wcpc = get_probdist_per_class(word_count_per_class)
data_pd = get_probdist_per_class(data[vocabs])


In [43]:
wcpc.shape

(10, 10473)

In [44]:
data_pd.shape

(2246, 10473)

In [54]:
def get_class_prob(wcpc, data):
    ret = np.dot(wcpc, data.T)
    tmp = []
    ret = ret.reshape(2246, 10)
    for rows in ret:
        sum = np.sum(rows)
        tmp.append(rows/sum)
    return tmp

prob = get_class_prob(wcpc, data_pd)

In [61]:
newz = []
for p in prob:
    s = multinomial.rvs(n=1,p=p,size=1)
    newz.append(np.argmax(s[0]))
newz

[5,
 8,
 8,
 1,
 9,
 8,
 5,
 2,
 6,
 5,
 0,
 5,
 5,
 9,
 5,
 1,
 7,
 3,
 5,
 7,
 9,
 4,
 0,
 3,
 1,
 5,
 7,
 0,
 0,
 2,
 9,
 6,
 8,
 4,
 4,
 7,
 1,
 4,
 8,
 7,
 7,
 1,
 5,
 9,
 5,
 1,
 3,
 1,
 3,
 4,
 2,
 3,
 6,
 8,
 4,
 3,
 4,
 5,
 8,
 3,
 4,
 9,
 3,
 8,
 8,
 6,
 7,
 5,
 2,
 1,
 5,
 8,
 8,
 8,
 8,
 1,
 6,
 9,
 8,
 9,
 8,
 6,
 6,
 6,
 9,
 6,
 7,
 9,
 7,
 9,
 6,
 9,
 8,
 3,
 6,
 8,
 3,
 5,
 6,
 3,
 9,
 4,
 5,
 2,
 3,
 4,
 4,
 8,
 6,
 2,
 8,
 4,
 5,
 5,
 7,
 5,
 8,
 3,
 0,
 1,
 3,
 8,
 1,
 2,
 4,
 5,
 4,
 1,
 5,
 3,
 5,
 5,
 3,
 8,
 5,
 0,
 0,
 2,
 6,
 0,
 8,
 9,
 5,
 3,
 6,
 9,
 3,
 9,
 0,
 8,
 7,
 5,
 2,
 4,
 5,
 6,
 9,
 6,
 5,
 4,
 5,
 9,
 0,
 6,
 3,
 9,
 1,
 0,
 3,
 1,
 2,
 0,
 7,
 6,
 1,
 4,
 3,
 1,
 4,
 5,
 9,
 0,
 6,
 3,
 4,
 4,
 4,
 0,
 1,
 5,
 1,
 5,
 5,
 1,
 5,
 3,
 9,
 9,
 3,
 0,
 8,
 2,
 8,
 8,
 8,
 4,
 9,
 8,
 4,
 8,
 9,
 6,
 5,
 4,
 2,
 7,
 0,
 5,
 3,
 2,
 2,
 3,
 4,
 1,
 0,
 3,
 4,
 0,
 9,
 2,
 8,
 1,
 6,
 0,
 7,
 7,
 1,
 3,
 5,
 7,
 7,
 6,
 5,
 9,
 5,
 5,
 1,
 1,
 2,
 3,
